In [81]:
import librosa.display
import librosa.util
import matplotlib.pyplot as plt
from IPython.display import Audio
import os
import scipy
from scipy.io import wavfile
import numpy as np
import utils.synthesis_util as su
import utils.model_util as mu
from utils.metrics import Metrics
from pesq_lib.pypesq import pypesq

## Examine dataset

In [99]:
audio_files = su.load_files()
id_audio=69

### Clean audio

In [100]:
y_clean,sr = su.get_audio(audio_id=id_audio,audio_files=audio_files)
Audio(y_clean,rate=sr)

## Noisy audio

In [101]:
y_noise, sr = su.get_noisy_audio(audio_id=id_audio,audio_files=audio_files, noise_factor=0.05)
Audio(y_noise,rate=sr)

0.40420088
0.009747084174796947 0.024114455498785627


In [85]:
win = scipy.signal.hamming(256,"periodic")
magnitude,phase = su.audio_to_sttft(y_noise,win)

In [86]:
# 

In [87]:
predictors = su.get_predictors(magnitude)

In [88]:
predictors = np.array(predictors)
predictors.shape

(462, 129, 8)

## Load module

In [89]:
model = mu.load_model(n_features=129,n_segments=8,model_to_test="Baseline_Convolutional_time_filtering/0513_154624", type='conv_time')
model_new = mu.load_model(n_features=129,n_segments=8,model_to_test="Baseline_Convolutional/0520_122048", type='conv')

## Denoise

In [90]:
length = len(y_noise)

In [91]:
sample = su.obtain_sample(predictors)

In [92]:
# print(model)
audio_rec = su.denoise_audio(model_new,sample,phase,win,length)
print(audio_rec.shape)
Audio(audio_rec,rate=sr)

(29841,)


# Calculate metrics

In [93]:
magnitude_clean,phase_clean = su.audio_to_sttft(y_clean,win)
magnitude_noise,phase_noise = su.audio_to_sttft(y_noise,win)
magnitude_denoised,phase_denoised = su.audio_to_sttft(audio_rec,win)
print(f"SNR_db for clean: {10*np.log10(Metrics.DER_SNR(Metrics.spectral_flux(magnitude_clean, sr)))}")
print(f"SNR_db for noise: {10*np.log10(Metrics.DER_SNR(Metrics.spectral_flux(magnitude_noise, sr)))}")
print(f"SNR_db for denoised: {10*np.log10(Metrics.DER_SNR(Metrics.spectral_flux(magnitude_denoised, sr)))}")


SNR_db for clean: 32.60752677528472
SNR_db for noise: 34.120934565486046
SNR_db for denoised: 32.91420041887038


In [94]:
print(sr)

8000.0


In [95]:
print(f"PESQ value with narrowband (nb) considering original signal and original+noise: {pypesq(sr, y_clean, y_noise, 'nb')}")
print(f"PESQ value with narrowband (nb) considering original signal and reconstructed signal: {pypesq(sr, y_clean, audio_rec, 'nb')}")

PESQ value with narrowband (nb) considering original signal and original+noise: 1.8767606019973755
PESQ value with narrowband (nb) considering original signal and reconstructed signal: 1.9355688095092773


### Aggregated metrics

In [96]:
import random

N = 10
indices = [random.randrange(0, len(audio_files)) for _ in range(N)]
print(len(audio_files))
print(indices)

11572
[5933, 4550, 9959, 3666, 10670, 4129, 9586, 6606, 5703, 5268]


In [97]:
#Metrics.aggregate_metric_check(audio_files, model, indices=indices)

In [98]:
#Metrics.aggregate_metric_check(audio_files, model_new, indices=indices)